In [1]:
%config IPCompleter.greedy=True

# Guardado de un archivo excel a una tabla

<ul>
    <li>Se leera un archivo de excel</li>
    <li>Se identificara el id del nombre de la estacion. Para esto se hara una consulta a la base de datos</li>
    <li>Se reemplaza la columna "nombre_estacion" por una que tendra el id correspondiente</li>
    <li>Se guardara la tabla en la base de datos</li>
</ul>

In [12]:
#Bibliotecas que se usaran.
import pandas as pd
import glob
import sqlalchemy

In [13]:
#Biblioteca que se usara para hacer match entre palabras
from fuzzywuzzy import process

In [14]:
#Se crea una ruta de conexion a la base de datos llamada clima, la cual se encuentra en el servidor DESKTOP-HQI9VLS
#Utilizamos la autentificacion de windows para conectarnos a este.
server = 'DESKTOP-HQI9VLS'
database = 'clima'
engine = sqlalchemy.create_engine('mssql+pyodbc://'+server+'/'+database+'?trusted_connection=yes&driver=SQL+Server')

In [15]:
#importamos los tipos para no tener para facilitar su uso
from sqlalchemy.types import VARCHAR,Float,Integer

In [16]:
#Obtenemos las rutas de los archivos xls y creamos un dataframe para guardarlas

#Esta ya no se hace
rutas = []
for i in glob.glob("Puebla/*.xls"):
    rutas.append(i)
datos = {'Rutas' : rutas}
rutas = pd.DataFrame(datos)
rutas.to_csv("Rutas - faltantes.csv")

In [19]:
#Leemos el archivo de rutas que faltan por agregar a la base de datos
rutas = pd.read_csv("Rutas - faltantes.csv")
rutas = rutas.drop(['Unnamed: 0'], axis=1)
rutas.head()

,Rutas
0,Puebla\huauchi ab14.xls
1,Puebla\huauchi ab15.xls
2,Puebla\huauchi ag14.xls
3,Puebla\huauchi d14.xls
4,Puebla\huauchi e13.xls


In [20]:
#Leemos el archivo de rutas que faltan por agregar a la base de datos
rutas = pd.read_csv("Rutas - faltantes.csv")
rutas = rutas.drop(['Unnamed: 0'], axis=1)
#Obtenemos la primera ruta y lo eliminamos del DF
r = rutas['Rutas'][0]
rutas = rutas.drop(0, axis=0)
#Se obitiene el catalogo
catalogo = pd.read_sql("SELECT id,nombre from CatalogoCorto", con=engine)
#Se pasa a una lista los nombres de las estaciones del catalogo
listaNombres = catalogo['nombre'].values
#Se abre el archivo de la ruta r
datos = pd.read_excel(r)
#Obtenemos el nombre de la estacion
nombreEstacion = datos["nombre_estacion"][0]
#Buscamos la palabra(de listaNombres) con mas similitud a nombreEstacion y la guardamos en masALto
masAlto, porcentaje = process.extractOne(nombreEstacion,listaNombres)
#Consultamos su id y obtenemos su valor
identificador = (catalogo[catalogo['nombre']==masAlto])['id'].values[0]
#Creamos una columna id_estado donde guardaremos la variable identificador
datos['id_estado'] = datos['nombre_estacion'].map(lambda x: identificador)
#Eliminamos la columna nombre estacion
datos = datos.drop(['nombre_estacion'], axis=1)
#Ordenamos las columnas
columnasOrdenadas = ['id_estado','fecha', 'DirViento', 'DirRafaga', 'RapViento', 'RapRafaga','TempAire', 
                     'HumRelativa', 'PresBarometric', 'Precipitacion', 'RadSolar']
guardar = pd.DataFrame()
guardar[columnasOrdenadas] = datos[columnasOrdenadas]
#Guardamos a la base de datos
guardar.to_sql(name='Datos', con = engine, index=False, if_exists= 'append')


#Se guardan las rutas que faltan por agregar
rutas.to_csv("Rutas - faltantes.csv")

#Si se quiere solo seleccionar algunas columnas para guardarlas en la base de datos se escifican en colBase
#Y el nombre en el archivo de excel en colExcel
#guardar = pd.DataFrame()
#guardar['id_estado'] = datos['nombre_estacion'].map(lambda x: identificador)
#colBase = ['Temperatura','Precipitacion','Humedad']
#colExcel = ['TempAire','Precipitacion','HumRelativa']
#guardar[colBase] = datos[colExcel]

